In [106]:
from datascience import *
import numpy as np
import pandas as pd
import codecs
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import math
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
import catboost as cb
import time

In [94]:
data   = pd.read_csv('PJT002_train.csv', encoding = 'UTF-8')
val    = pd.read_csv('PJT002_validation.csv', encoding = 'UTF-8')
test   = pd.read_csv('PJT002_test.csv', encoding = 'UTF-8')
sub    = pd.read_csv('PJT002_submission.csv', encoding = 'UTF-8')

C:\Users\ybahn\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [95]:
# 결측된 기온을 우리나라 평균치로 대체
data['tmprtr']  = data['tmprtr'].fillna(12.5)
data['prcpttn'] = data['prcpttn'].fillna(3.84)
data['hmdt']    = data['hmdt'].fillna(70.0)
data['wnd_spd'] = data['wnd_spd'].fillna(1.53)

val['tmprtr']   = val['tmprtr'].fillna(12.5)
val['prcpttn']  = val['prcpttn'].fillna(3.84)
val['hmdt']     = val['hmdt'].fillna(70.0)
val['wnd_spd']  = val['wnd_spd'].fillna(1.53)

test['tmprtr']  = test['tmprtr'].fillna(12.5)
test['prcpttn'] = test['prcpttn'].fillna(3.84)
test['hmdt']    = test['hmdt'].fillna(70.0)
test['wnd_spd'] = test['wnd_spd'].fillna(1.53)

In [97]:
col_1                 = ['bldng_us', 'bldng_archtctr', 'bldng_ar', 'bldng_us_clssfctn', 'tmprtr', 'prcpttn',
                         'wnd_spd', 'wnd_drctn', 'hmdt', 'fr_wthr_fclt_dstnc', 'mlt_us_yn', 'fr_wthr_fclt_in_100m',
                         'no_tbc_zn_dstnc','trgt_crtr', 'fr_fghtng_fclt_spcl_css_5_yn', 'fr_fghtng_fclt_spcl_css_6_yn', 
                         'us_yn', 'dngrs_thng_yn', 'slf_fr_brgd_yn', 'blk_dngrs_thng_mnfctr_yn']

data_x                = data[col_1]
val_x                 = val[col_1]
test_x                = test[col_1]


col_2                 = ['lw_13101010', 'lw_13101110','lw_13101210', 'lw_13101211','lw_13101310',
                         'lw_13101410','lw_13111010','lw_13111110',
                         'lw_13121010','lw_13121011','lw_13131010','lw_13131110','lw_13141010','lw_13141011']

data_x                = pd.concat([data_x, data[col_2]], axis = 1)
data_x['lw_13101010'] = 3
data_x['lw_13101110'] = 3

val_x                 = pd.concat([val_x, val[col_2]], axis = 1)
val_x['lw_13101010']  = 3
val_x['lw_13101110']  = 3

test_x                = pd.concat([test_x, test[col_2]], axis = 1)
test_x['lw_13101010'] = 3
test_x['lw_13101110'] = 3







data_x[col_2]         = round(data_x[col_2])
val_x[col_2]          = round(val_x[col_2])
test_x[col_2]         = round(test_x[col_2])

data_x                = data_x.fillna(0)
val_x                 = val_x.fillna(0)
test_x                = test_x.fillna(0)



data_y                = pd.DataFrame(data['fr_yn'].map({'N':0, 'Y':1}))
val_y                 = pd.DataFrame(val['fr_yn'].map({'N':0, 'Y':1}))

In [98]:
year = []

data['dt_of_athrztn'][32635] = 2002

for i in range(0 , len(data)):
    if math.isnan(float(data['dt_of_athrztn'][i])):
        year = year + ['1990']
    elif int(str(data['dt_of_athrztn'][i])[:4]) > 1930 and int(str(data['dt_of_athrztn'][i])[:4]) < 1940 :
        year = year + ['1930']
    elif int(str(data['dt_of_athrztn'][i])[:4]) > 1940 and int(str(data['dt_of_athrztn'][i])[:4]) < 1950 :
        year = year + ['1940']
    elif int(str(data['dt_of_athrztn'][i])[:4]) > 1950 and int(str(data['dt_of_athrztn'][i])[:4]) < 1960 :
        year = year + ['1950']
    elif int(str(data['dt_of_athrztn'][i])[:4]) > 1960 and int(str(data['dt_of_athrztn'][i])[:4]) < 1970 :
        year = year + ['1960']
    elif int(str(data['dt_of_athrztn'][i])[:4]) > 1970 and int(str(data['dt_of_athrztn'][i])[:4]) < 1980 :
        year = year + ['1970']
    elif int(str(data['dt_of_athrztn'][i])[:4]) > 1980 and int(str(data['dt_of_athrztn'][i])[:4]) < 1990 :
        year = year + ['1980']
    elif int(str(data['dt_of_athrztn'][i])[:4]) > 1990 and int(str(data['dt_of_athrztn'][i])[:4]) < 2000 :
        year = year + ['1990']
    elif int(str(data['dt_of_athrztn'][i])[:4]) > 2000 and int(str(data['dt_of_athrztn'][i])[:4]) < 2010 :
        year = year + ['2000']
    elif int(str(data['dt_of_athrztn'][i])[:4]) > 2010 and int(str(data['dt_of_athrztn'][i])[:4]) < 2020 :
        year = year + ['2010']
    else :
        year = year + ['1990']
data_x['year'] = year

        
        
year = []

for i in range(0 , len(val)):
    if math.isnan(float(val['dt_of_athrztn'][i])):
        year = year + ['1990']
    elif int(str(val['dt_of_athrztn'][i])[:4]) > 1930 and int(str(val['dt_of_athrztn'][i])[:4]) < 1940 :
        year = year + ['1930']
    elif int(str(val['dt_of_athrztn'][i])[:4]) > 1940 and int(str(val['dt_of_athrztn'][i])[:4]) < 1950 :
        year = year + ['1940']
    elif int(str(val['dt_of_athrztn'][i])[:4]) > 1950 and int(str(val['dt_of_athrztn'][i])[:4]) < 1960 :
        year = year + ['1950']
    elif int(str(val['dt_of_athrztn'][i])[:4]) > 1960 and int(str(val['dt_of_athrztn'][i])[:4]) < 1970 :
        year = year + ['1960']
    elif int(str(val['dt_of_athrztn'][i])[:4]) > 1970 and int(str(val['dt_of_athrztn'][i])[:4]) < 1980 :
        year = year + ['1970']
    elif int(str(val['dt_of_athrztn'][i])[:4]) > 1980 and int(str(val['dt_of_athrztn'][i])[:4]) < 1990 :
        year = year + ['1980']
    elif int(str(val['dt_of_athrztn'][i])[:4]) > 1990 and int(str(val['dt_of_athrztn'][i])[:4]) < 2000 :
        year = year + ['1990']
    elif int(str(val['dt_of_athrztn'][i])[:4]) > 2000 and int(str(val['dt_of_athrztn'][i])[:4]) < 2010 :
        year = year + ['2000']
    elif int(str(val['dt_of_athrztn'][i])[:4]) > 2010 and int(str(val['dt_of_athrztn'][i])[:4]) < 2020 :
        year = year + ['2010']
    else :
        year = year + ['1990']
val_x['year']  = year     
        
        
year = []

for i in range(0 , len(test)):
    if math.isnan(float(test['dt_of_athrztn'][i])):
        year = year + ['1990']
    elif int(str(test['dt_of_athrztn'][i])[:4]) > 1930 and int(str(test['dt_of_athrztn'][i])[:4]) < 1940 :
        year = year + ['1930']
    elif int(str(test['dt_of_athrztn'][i])[:4]) > 1940 and int(str(test['dt_of_athrztn'][i])[:4]) < 1950 :
        year = year + ['1940']
    elif int(str(test['dt_of_athrztn'][i])[:4]) > 1950 and int(str(test['dt_of_athrztn'][i])[:4]) < 1960 :
        year = year + ['1950']
    elif int(str(test['dt_of_athrztn'][i])[:4]) > 1960 and int(str(test['dt_of_athrztn'][i])[:4]) < 1970 :
        year = year + ['1960']
    elif int(str(test['dt_of_athrztn'][i])[:4]) > 1970 and int(str(test['dt_of_athrztn'][i])[:4]) < 1980 :
        year = year + ['1970']
    elif int(str(test['dt_of_athrztn'][i])[:4]) > 1980 and int(str(test['dt_of_athrztn'][i])[:4]) < 1990 :
        year = year + ['1980']
    elif int(str(test['dt_of_athrztn'][i])[:4]) > 1990 and int(str(test['dt_of_athrztn'][i])[:4]) < 2000 :
        year = year + ['1990']
    elif int(str(test['dt_of_athrztn'][i])[:4]) > 2000 and int(str(test['dt_of_athrztn'][i])[:4]) < 2010 :
        year = year + ['2000']
    elif int(str(test['dt_of_athrztn'][i])[:4]) > 2010 and int(str(test['dt_of_athrztn'][i])[:4]) < 2020 :
        year = year + ['2010']
    else :
        year = year + ['1990']
        




test_x['year']   = year

C:\Users\ybahn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
dummi = ['trgt_crtr','bldng_archtctr', 'bldng_us','bldng_us_clssfctn', 'blk_dngrs_thng_mnfctr_yn', 'dngrs_thng_yn',
         'fr_fghtng_fclt_spcl_css_5_yn', 'fr_fghtng_fclt_spcl_css_6_yn', 'fr_wthr_fclt_in_100m',
         'mlt_us_yn', 'slf_fr_brgd_yn', 'us_yn', 'year']

data_x = pd.get_dummies(data = data_x, columns = dummi)
val_x  = pd.get_dummies(data = val_x, columns = dummi)
test_x = pd.get_dummies(data = test_x, columns = dummi)


In [100]:
# 없는 column 찾고 0 대입

yes = []
no  = []

for i in range(0,len(data_x.columns)):
    if data_x.columns[i] in list(val_x.columns):
        yes = yes + [data_x.columns[i]]
    else :
        no  = no + [data_x.columns[i]]

yes = []
no  = []        

for i in range(0,len(data_x.columns)):    
    if data_x.columns[i] in list(test_x.columns):
        yes = yes + [data_x.columns[i]]
    else:
        no  = no + [data_x.columns[i]]

        
for i in range(0, len(no)):
    if len(no) == 0 :
        break
    else :
        val_x[str(no[i])]  = 0
        test_x[str(no[i])] = 0
        
val_x  = val_x[list(data_x.columns)]
test_x = test_x[list(data_x.columns)]


# 모델 학습

In [101]:
# test set & train set 분할
X = data_x
y = data['fr_yn'].map({'N' : 0, 'Y' : 1})

# CatBoost

In [103]:
start            = time.time() # 시작 시간 지정
cb_dtrain        = cb.Pool(data = X, label = y) # 학습 데이터를 Catboost 모델에 맞게 변환
cb_param         = {'max_depth'     : 16, # 트리 깊이
                    'learning_rate' : 0.01, # Step Size
                    'n_estimators'  : 100, # Number of trees, 트리 생성 개수
                    'eval_metric'   : 'Accuracy', # 평가 척도
                    'loss_function' : 'MultiClass'} # 손실 함수, 목적 함수
cb_model         = cb.train(pool = cb_dtrain, params = cb_param) # 학습 진행
cb_model_predict = np.argmax(cb_model.predict(val_x), axis = 1) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측, 인덱스의 순서를 맞추기 위해 +1

cb_predict = pd.DataFrame()
cb_predict['fr_yn'] = np.array(cb_model_predict)

f1_score(val_y, cb_predict)

0:	learn: 0.8847954	total: 74.5ms	remaining: 7.37s
1:	learn: 0.8881569	total: 74.5ms	remaining: 7.37s
2:	learn: 0.8896434	total: 4.86s	remaining: 3m 55s
3:	learn: 0.8902008	total: 6.1s	remaining: 3m 15s
4:	learn: 0.8916705	total: 16.8s	remaining: 6m 39s
5:	learn: 0.8919407	total: 26.5s	remaining: 8m 18s
6:	learn: 0.8927516	total: 35.2s	remaining: 9m 5s
7:	learn: 0.8934779	total: 44.5s	remaining: 9m 45s
8:	learn: 0.8941198	total: 1m 7s	remaining: 12m 49s
9:	learn: 0.8939002	total: 1m 7s	remaining: 11m 17s
10:	learn: 0.8942381	total: 1m 17s	remaining: 11m 28s
11:	learn: 0.8940860	total: 1m 18s	remaining: 10m 27s
12:	learn: 0.8942381	total: 1m 27s	remaining: 10m 36s
13:	learn: 0.8944239	total: 1m 38s	remaining: 10m 48s
14:	learn: 0.8940692	total: 1m 47s	remaining: 10m 54s
15:	learn: 0.8943563	total: 1m 57s	remaining: 10m 57s
16:	learn: 0.8944070	total: 2m 6s	remaining: 10m 57s
17:	learn: 0.8947111	total: 2m 15s	remaining: 10m 54s
18:	learn: 0.8941029	total: 2m 15s	remaining: 10m 11s
19:	l

0.4947253546744271

In [54]:
#cb_model_predict = np.argmax(cb_model.predict(val_x, prediction_type = 'Probability'), axis = 1)

cb_model_predict = cb_model.predict(test_x, prediction_type = 'Probability')

cb_model_predict[0]

array([0.81156227, 0.18843773])

In [83]:
cb_model_predict = np.argmax(cb_model.predict(test_x), axis = 1)

temp          = pd.DataFrame()
temp['fr_yn'] = cb_model_predict

sub['fr_yn'] = temp['fr_yn'].map({0 : 'N', 1 : 'Y'})
sub.to_csv('dydqja95_화재예측과제_Submission.csv', encoding = 'UTF-8', index = False)

# Adaboost

In [102]:
ada_clf  = AdaBoostClassifier(n_estimators = 100, random_state = 0)
ada_clf.fit(X, y) 
ada_model_predict = ada_clf.predict(val_x)
print(("f1: %.2f" % (f1_score(val_y, ada_model_predict) * 100)), '%')

f1: 48.37 %


In [85]:
ada_model_predict = ada_clf.predict(test_x)

t = pd.DataFrame()
t['fr_yn'] = ada_model_predict

sub['fr_yn'] = t['fr_yn'].map({0 : 'N', 1 : 'Y'})
sub.to_csv('dydqja95_화재예측과제_Submission.csv', encoding = 'UTF-8', index = False)

# 모델 확인

In [88]:
cb_model_proba  = cb_model.predict(test_x, prediction_type = 'Probability')

ada_model_proba = ada_clf.predict_proba(test_x)

In [91]:
last       = ((cb_model_proba*0.7) + (ada_model_proba*1.3))/2

last_model = []

for i in range(0, len(cb_model_predict)):
    if last[i][0] > last[i][1]:
        last_model = np.append(last_model, 'N')
    
    else:
        last_model = np.append(last_model, 'Y')

last_model

array(['N', 'Y', 'N', ..., 'N', 'N', 'Y'], dtype='<U32')

In [92]:
sub['fr_yn'] = last_model
sub.to_csv('dydqja95_화재예측과제_Submission.csv', encoding = 'UTF-8', index = False)